In [ ]:
FULL_TRAINING = False

import numpy as np
import pandas as pd
from pynvml import *
import torch
from torch.utils.data import Dataset
from PIL import Image
from transformers import VisionEncoderDecoderModel, AutoTokenizer

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()


In [ ]:
model = None
tokenizer = None

if FULL_TRAINING:
    vision_hf_model = 'facebook/deit-base-distilled-patch16-384'
    nlp_hf_model = "hfl/chinese-macbert-base"
    
    # Reference: https://github.com/huggingface/transformers/issues/15823
    # initialize the encoder from a pretrained ViT and the decoder from a pretrained BERT model. 
    # Note that the cross-attention layers will be randomly initialized, and need to be fine-tuned on a downstream dataset
    model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(vision_hf_model, nlp_hf_model)
    tokenizer = AutoTokenizer.from_pretrained(nlp_hf_model)
else:
    trocr_model = 'models/epoch-1/'
    model = VisionEncoderDecoderModel.from_pretrained(trocr_model)
    tokenizer = AutoTokenizer.from_pretrained(trocr_model)     

In [ ]:
# from sklearn.model_selection import train_test_split

# train_df, test_df = train_test_split(df, test_size=0.1)
# train_df.reset_index(drop=True, inplace=True)
# test_df.reset_index(drop=True, inplace=True)

In [ ]:
import os
from os import path

import torch
from torch.utils.data import Dataset
import torchvision.io as io
import pandas as pd


class OCRDataset(Dataset):
    def __init__(self, dataset_dir, labels_dir, transform, processor, tokenizer, mode="train", max_target_length=32, device=None):
        self.dataset_dir = dataset_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.device = device
        self.processor = processor
        self.mode = mode
        self.max_target_length = max_target_length
        self.tokenizer = tokenizer
        self.df = self.build_df()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text
        file_name = self.df["file_name"][idx]
        text = self.df['text'][idx]
        # prepare image (i.e. resize + normalize)
        image = Image.open(path.join(self.dataset_dir, file_name)).convert("RGB")
        if self.mode == "train" and self.transform:
            image = self.transform(image)

        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        labels = self.tokenizer(text, padding="max_length",
                                stride=32,
                                truncation=True,
                                max_length=self.max_target_length).input_ids
        
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

    def build_df(self):
        li = []
        for root, dirs, files in os.walk(self.labels_dir):
            for file in files:  # Loop through the dataset tsvfiles
                if not file.endswith(".tsv"):
                    continue

                print(f"Processing {file}")
                li.append(pd.read_table(path.join(root, file),
                          names=["file_name", "text"]))

        return pd.concat(li, axis=0, ignore_index=True)

In [ ]:
from transformers import TrOCRProcessor
from data_aug import build_data_aug
from torch.utils.data import Subset, random_split

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
dataset_dir = 'dataset/data'
max_length = 64

train_dataset = OCRDataset(
    dataset_dir=dataset_dir,
    labels_dir="dataset/labels/train",
    tokenizer=tokenizer,
    processor=processor,
    mode="train",
    transform=build_data_aug((64, 1024), "train"),
    max_target_length=max_length
)

# Define the number of samples to keep in eval dataset
num_samples = 100

eval_dataset = OCRDataset(
    dataset_dir=dataset_dir,
    labels_dir="dataset/labels/test",
    tokenizer=tokenizer,
    processor=processor,
    mode="eval",
    transform=None,
    max_target_length=max_length
)

# Create a random subset of the dataset
subset_indices = torch.randperm(len(eval_dataset))[:num_samples]
eval_dataset = Subset(eval_dataset, subset_indices.tolist())

print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

In [ ]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = tokenizer.sep_token_id
model.config.max_length = max_length
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    fp16=True,
    learning_rate=4e-5,
    output_dir="./checkpoints",
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="steps",
    save_total_limit=5,
    save_steps=10000,
    eval_steps=10000,
    resume_from_checkpoint="./checkpoints/"
)

문자 오류율(CER; Character Error Rate)과 단어 오류율(WER; Word Error Rate) 지표로 모델을 평가합니다.

In [ ]:
import evaluate
cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    labels_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=labels_str)
    wer = wer_metric.compute(predictions=pred_str, references=labels_str)

    return {"cer": cer, "wer": wer}

In [ ]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)

In [ ]:
print_gpu_utilization()

In [ ]:
result = trainer.train("models/epoch-1")

In [ ]:
if model:
    del model
torch.cuda.empty_cache()

In [ ]:
if FULL_TRAINING:
    steps = []
    losses = []
    for obj in trainer.state.log_history:
        if obj.get("step") and obj.get("loss"):
            steps.append(obj['step'])
            losses.append(obj['loss'])

    import matplotlib.pyplot as plt
    f = plt.figure(figsize=(12,6))
    plt.plot(steps, losses)
    plt.xlabel('step')
    plt.ylabel('training loss')
    plt.show()    

In [ ]:
import random
SCALER_NAME = "scaler.pt"
SCHEDULER_NAME = "scheduler.pt"
OPTIMIZER_NAME = "optimizer.pt"

output_dir = "./models/epoch-1"

trainer.save_model(output_dir=output_dir)
trainer.save_state()

torch.save(trainer.scaler.state_dict(), os.path.join(output_dir, SCALER_NAME))
torch.save(trainer.optimizer.state_dict(), os.path.join(output_dir, OPTIMIZER_NAME))
torch.save(trainer.scaler.state_dict(), os.path.join(output_dir, SCALER_NAME))
torch.save(trainer.lr_scheduler.state_dict(), os.path.join(output_dir, SCHEDULER_NAME))
# Single gpu
rng_states = {
    "python": random.getstate(),
    "numpy": np.random.get_state(),
    "cpu": torch.random.get_rng_state(),
    "cuda": torch.cuda.random.get_rng_state_all()
}
torch.save(rng_states, os.path.join(output_dir, "rng_state.pth"))

<br>

## Evaluation and Inference
---


In [ ]:
from torch.utils.data import Subset, random_split

# Define the number of samples you want to keep
num_samples = 100

# Create a random subset of the dataset
subset_indices = torch.randperm(len(eval_dataset))[:num_samples]
subset = Subset(eval_dataset, subset_indices.tolist())


model.eval()
with torch.no_grad():
    eval_result = trainer.evaluate(subset, max_length=64)    

In [ ]:
eval_result

In [ ]:
import glob
sample_img_paths = glob.glob('sample_imgs/*.png')
img_idx = np.random.randint(len(sample_img_paths))
image = Image.open(sample_img_paths[img_idx]).convert("RGB")
#img_idx = np.random.randint(len(eval_dataset))
#image = Image.open(eval_dataset.dataset_dir + train_df['file_name'][img_idx])
image

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
device = torch.device('cuda')

pixel_values = (processor(image, return_tensors="pt").pixel_values).to(device)
generated_ids = model.generate(pixel_values)
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0] 
generated_ids, generated_text

In [ ]:
labels = torch.tensor([ 101,  101, 3330, 3152, 3232, 2821, 6624, 1092,  749,  511,  800, 1440,
          6401, 1920, 2157, 6432, 8038, 1157, 2798, 2769,  794, 3178,  102])
# generated_text = tokenizer.convert_ids_to_tokens(labels, skip_special_tokens=True)
generated_text = tokenizer.decode(labels, clean_up_tokenization_spaces=True, skip_special_tokens=False)
generated_text

<br>

## Clean up
---


In [ ]:
!rm -rf model

## (Optional) Upload to Hugging Face Hub

In [ ]:
# !pip install huggingface_hub

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
# # repo
# MODEL_SAVE_REPO = '[YOUR-REPO]'
# HUGGINGFACE_AUTH_TOKEN = '[YOUR-TOKEN]' # https://huggingface.co/settings/token

# # Push to huggingface-hub
# model.push_to_hub(
#     MODEL_SAVE_REPO,
#     use_temp_dir=True,
#     use_auth_token=HUGGINGFACE_AUTH_TOKEN
# )

# tokenizer.push_to_hub(
#     MODEL_SAVE_REPO,
#     use_temp_dir=True,
#     use_auth_token=HUGGINGFACE_AUTH_TOKEN
# )